In [1]:
from __future__ import absolute_import
from __future__ import print_function
import os
import glob
import random
import numpy as np
from keras import optimizers
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.layers.wrappers import TimeDistributed
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.layers import Input, InputLayer
from keras.layers.core import Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.applications import imagenet_utils

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
videoFiles = glob.glob('../deep-dataset/numpys/*.npy')
mosFiles = [i for i in videoFiles if 'mos' in i]
videoFiles = [i for i in videoFiles if 'mos' not in i]

In [38]:
def myGenerator():
    while True:
        index_list = random.sample(range(1, 405), 2)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            #print(i, index_list)
            f = videoFiles[i]
            s = f[:-4]+'_mos.npy'
            a = np.load(f)
            b = np.load(s)
            alldata_x.append(a)
            alldata_y.append(b)
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        #print(alldata_x.shape, alldata_y.shape)
        yield alldata_x, alldata_y
# x = myGenerator()
# xtrain, ytrain = next(x)
# print('xtrain shape:',xtrain.shape)
# print('ytrain shape:',ytrain.shape)

In [29]:
print(len(videoFiles))
for i in videoFiles:
    print(np.load(i).shape)
    break

405
(200, 68, 120, 3)


In [32]:
height = 68
width = 120
input_shape=(200, height, width, 3)

In [36]:
def mySegNet(input_shape):
    base_model  = MobileNet(input_shape=(224,224,3), include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs=base_model.input, outputs=x)
    
    model = Sequential();
    #model.add(InputLayer(input_shape=input_shape))
    model.add(TimeDistributed(cnn_model, input_shape=input_shape))
    model.add(TimeDistributed(Flatten()))
    #model.add(cnn_model)
    #model.add(Flatten())
    
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(5, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy')
    print(model.summary())
    return model 
#mySegNet(input_shape)

In [ ]:
model = mySegNet(input_shape)
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=True,
                   steps_per_epoch=15, epochs=50)
# model.save('model1.h5')
# model.save_weights('model_weights1.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_13 (TimeDis (None, 200, 1024)         3228864   
_________________________________________________________________
time_distributed_14 (TimeDis (None, 200, 1024)         0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 50)                215000    
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 255       
Total params: 3,444,119
Trainable params: 3,422,231
Non-trainable params: 21,888
_________________________________________________________________
None
Epoch 1/50
15/15 [==============================] - 40s 3s/step - loss: 1.6532
Epoch 2/50
15/15 [==============================] - 19s 1s/step - loss: 1.4796
Epoch 3/50
15/15 [==============================] - 18s 1s/step - loss: 1.4465
Epoch 4/50
15/15 